In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

from datasets.Era5 import Era5
from datasets.Dem import Dem
from datasets.Sentinel3 import Sentinel3
from datasets.Sentinel5 import Sentinel5
from datasets.LandCover import LandCover
from datasets.CollectionDataset import CollectionDataset
from utils import process_images, get_day_of_year_and_day_of_week

from presto.presto import Encoder, Decoder, Presto
from PixelTimeseries import PixelTimeSeries
from PrestoMaskedLanguageModel import make_mask

MASK_STRATEGIES = (
    "group_bands",
    "random_timesteps",
    "chunk_timesteps",
    "random_combinations",
)

dataset = PixelTimeSeries(num_timesteps=5, input_data_path = 'data_small_file.pt')

train_dataloader = DataLoader(
        dataset,
        batch_size=64,
        shuffle=False,
    )
last_x = None
last_hard_mask = None
for x, hard_mask, latlons, day_of_year, day_of_week in train_dataloader:
    soft_mask = make_mask(x, hard_mask, MASK_STRATEGIES[2], 0.1)
    last_x = x
    last_hard_mask = hard_mask


Num timesteps:  5
Num timesteps to mask:  1
timesteps_to_mask [4]
n_timesteps_not_masked tensor(4.)
timesteps_to_mask [0]
n_timesteps_not_masked tensor(5.)
timesteps_to_mask [3]
n_timesteps_not_masked tensor(5.)
timesteps_to_mask [1]
n_timesteps_not_masked tensor(5.)
timesteps_to_mask [3]
n_timesteps_not_masked tensor(3.)
timesteps_to_mask [2]
n_timesteps_not_masked tensor(5.)
timesteps_to_mask [2]
n_timesteps_not_masked tensor(5.)
timesteps_to_mask [1]
n_timesteps_not_masked tensor(10.)
timesteps_to_mask [4]
n_timesteps_not_masked tensor(4.)
timesteps_to_mask [3]
n_timesteps_not_masked tensor(5.)
timesteps_to_mask [1]
n_timesteps_not_masked tensor(10.)
timesteps_to_mask [3]
n_timesteps_not_masked tensor(5.)
timesteps_to_mask [4]
n_timesteps_not_masked tensor(4.)
timesteps_to_mask [1]
n_timesteps_not_masked tensor(10.)
timesteps_to_mask [3]
n_timesteps_not_masked tensor(5.)
timesteps_to_mask [2]
n_timesteps_not_masked tensor(5.)
timesteps_to_mask [4]
n_timesteps_not_masked tensor(3.)
t

In [2]:
import numpy as np
for i in range(64):
    print( np.logical_and(soft_mask[i], last_hard_mask[i]).sum())
    size = last_x[i].shape[0] * last_x[i].shape[1]
    ratio = soft_mask[i].sum() / ( size - hard_mask[i].sum())
    print( ratio )
    print( soft_mask[i].sum() , soft_mask[i].shape , last_x[i].shape , last_hard_mask[i].sum() ,"\n")

tensor(0)
tensor(0.2180)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(19.) 

tensor(0)
tensor(0.2222)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(23.) 

tensor(0)
tensor(0.2335)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(33.) 

tensor(0)
tensor(0.2180)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(19.) 

tensor(0)
tensor(0.2222)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(23.) 

tensor(0)
tensor(0.2335)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(33.) 

tensor(0)
tensor(0.2190)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(20.) 

tensor(0)
tensor(0.2222)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(23.) 

tensor(0)
tensor(0.2312)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(31.) 

tensor(0)
tensor(0.2190)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(20.) 

tensor(0)
tensor(0.2222)
tensor(46) torch.Size([5, 46]) torch.Size([5, 46]) tensor(23.) 

tensor(0)


IndexError: index 55 is out of bounds for dimension 0 with size 55

In [ ]:
from datasets.CollectionDataset import BANDS, BANDS_GROUPS_IDX, BAND_EXPANSION
BAND_EXPANSION
BANDS_GROUPS_IDX